# Olympic medal Dataset analysis

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from collections import Counter

In [16]:
# Reading csv ( without using pd.read_csv()) and convert it in a dictionary
with open('olympic_1896.csv') as file :
    reader = csv.reader(file)  
    header = next(reader)
    dict_olympics = { h:[] for h in header }
    for line in reader :
        for key,value in zip(header,line) :
            dict_olympics[key].append(value)
    
print(dict_olympics.keys())
        
    

dict_keys(['City', 'Edition', 'Sport', 'Discipline', 'Athlete', 'NOC', 'Gender', 'Event', 'Event_gender', 'Medal'])


In [123]:
# Create dataFrame to Manage easier the data
medals = pd.DataFrame(dict_olympics)
medals.head()

,Athlete,City,Discipline,Edition,Event,Event_gender,Gender,Medal,NOC,Sport
0,"HAJOS, Alfred",Athens,Swimming,1896,100m freestyle,M,Men,Gold,HUN,Aquatics
1,"HERSCHMANN, Otto",Athens,Swimming,1896,100m freestyle,M,Men,Silver,AUT,Aquatics
2,"DRIVAS, Dimitrios",Athens,Swimming,1896,100m freestyle for sailors,M,Men,Bronze,GRE,Aquatics
3,"MALOKINIS, Ioannis",Athens,Swimming,1896,100m freestyle for sailors,M,Men,Gold,GRE,Aquatics
4,"CHASAPIS, Spiridon",Athens,Swimming,1896,100m freestyle for sailors,M,Men,Silver,GRE,Aquatics


In [128]:
#Statistics of the dataframe
medals.describe()

,Athlete,City,Discipline,Edition,Event,Event_gender,Gender,Medal,NOC,Sport
count,29216,29216,29216,29216,29216,29216,29216,29216,29216,29216
unique,21413,22,56,26,577,3,2,3,138,42
top,"LATYNINA, Larisa",Athens,Athletics,2008,football,M,Men,Gold,USA,Aquatics
freq,18,2149,3448,2042,1387,20067,21721,9850,4335,3828


## Top 15 countries ranked by total number of medals

In [132]:
# Select the 'NOC' column of medals: country_names
country_names = medals['NOC']
# Count the number of medals won by each country: medal_counts
medal_counts = country_names.value_counts()

# Print top 15 countries ranked by medals
print(medal_counts.head(15))




USA    4335
URS    2049
GBR    1594
FRA    1314
ITA    1228
GER    1211
AUS    1075
HUN    1053
SWE    1021
GDR     825
NED     782
JPN     704
CHN     679
RUS     638
ROU     624
Name: NOC, dtype: int64


## Count medals by type

In [131]:
# Construct the pivot table: counted
counted = medals.pivot_table(index='NOC',values='Athlete',columns='Medal',aggfunc='count')

# Create the new column: counted['totals']
counted['totals'] = counted.sum(axis='columns')

# Sort counted by the 'totals' column
counted = counted.sort_values('totals', ascending=False)

# Print the top 15 rows of counted
print(counted.head(15))

Medal  Bronze    Gold  Silver  totals
NOC                                  
USA    1052.0  2088.0  1195.0  4335.0
URS     584.0   838.0   627.0  2049.0
GBR     505.0   498.0   591.0  1594.0
FRA     475.0   378.0   461.0  1314.0
ITA     374.0   460.0   394.0  1228.0
GER     454.0   407.0   350.0  1211.0
AUS     413.0   293.0   369.0  1075.0
HUN     345.0   400.0   308.0  1053.0
SWE     325.0   347.0   349.0  1021.0
GDR     225.0   329.0   271.0   825.0
NED     320.0   212.0   250.0   782.0
JPN     270.0   206.0   228.0   704.0
CHN     193.0   234.0   252.0   679.0
RUS     240.0   192.0   206.0   638.0
ROU     282.0   155.0   187.0   624.0


## Understanding the columns labels

### Selecting types of gender columns

In [136]:
# Select columns: ev_gen
ev_gen =  medals[['Event_gender','Gender']]

# Drop duplicate pairs: ev_gen_uniques
ev_gen_uniques = ev_gen.drop_duplicates()

# Print ev_gen_uniques
print(ev_gen_uniques)

      Event_gender Gender
0                M    Men
348              X    Men
416              W  Women
639              X  Women
23675            W    Men


### Finding possible errors

In [137]:
# Group medals by the two columns: medals_by_gender
medals_by_gender = medals.groupby(['Event_gender','Gender'])

# Create a DataFrame with a group count: medal_count_by_gender
medal_count_by_gender = medals_by_gender.count()

# Print medal_count_by_gender
print(medal_count_by_gender)

                     Athlete   City  Discipline  Edition  Event  Medal    NOC  \
Event_gender Gender                                                             
M            Men       20067  20067       20067    20067  20067  20067  20067   
W            Men           1      1           1        1      1      1      1   
             Women      7277   7277        7277     7277   7277   7277   7277   
X            Men        1653   1653        1653     1653   1653   1653   1653   
             Women       218    218         218      218    218    218    218   

                     Sport  
Event_gender Gender         
M            Men     20067  
W            Men         1  
             Women    7277  
X            Men      1653  
             Women     218  


### Locating suspiscious data

In [139]:
# Boolean Series: sus
sus = (medals['Event_gender'] == 'W') & (medals['Gender'] == 'Men')

# The suspicious row: suspect
suspect = medals[sus]

# Print suspect
print(suspect)

                 Athlete    City Discipline Edition     Event Event_gender  \
23675  CHEPCHUMBA, Joyce  Sydney  Athletics    2000  marathon            W   

      Gender   Medal  NOC      Sport  
23675    Men  Bronze  KEN  Athletics  


## Constructing alternative country rankings

### Which countries won medals in the most distinct sports ?

In [ ]:
# Group medals by 'NOC': country_grouped
country_grouped = medals.groupby('NOC')

# Compute the number of distinct sports in which each country won medals: Nsports
Nsports = country_grouped['Sport'].nunique()

# Sort the values of Nsports in descending order
Nsports = Nsports.sort_values(ascending=False)

# Print the top 15 rows of Nsports
print(Nsports.head(15))

## Counting USA vs. USSR Cold War Olympic Sports

In [142]:
# Extract all rows for which the 'Edition' is between 1952 & 1988: during_cold_war
during_cold_war = (medals.Edition >= '1952') & (medals.Edition <= '1988')

# Extract rows for which 'NOC' is either 'USA' or 'URS': is_usa_urs
is_usa_urs = medals.NOC.isin(['USA','URS'])

# Use during_cold_war and is_usa_urs to create the DataFrame: cold_war_medals
cold_war_medals = medals.loc[during_cold_war & is_usa_urs]

# Group cold_war_medals by 'NOC'
country_grouped = cold_war_medals.groupby('NOC')

# Create Nsports
Nsports = country_grouped['Sport'].nunique().sort_values(ascending=False)

# Print Nsports
print(Nsports)


NOC
URS    21
USA    20
Name: Sport, dtype: int64


### Which country, the USA or the USSR, won the most medals consistently over the Cold War period ?

In [146]:
# Create the pivot table: medals_won_by_country
medals_won_by_country = medals.pivot_table(index='Edition',columns='NOC',values='Athlete',aggfunc='count')

# Slice medals_won_by_country: cold_war_usa_usr_medals
cold_war_usa_usr_medals = medals_won_by_country.loc['1952':'1988', ['USA',
'URS']]
print(cold_war_usa_usr_medals.head(10))

# Create most_medals 
most_medals = cold_war_usa_usr_medals.idxmax(axis='columns')

# Print most_medals.value_counts()
print(most_medals.value_counts())


NOC        USA    URS
Edition              
1952     130.0  117.0
1956     118.0  169.0
1960     112.0  169.0
1964     150.0  174.0
1968     149.0  188.0
1972     155.0  211.0
1976     155.0  285.0
1980       NaN  442.0
1984     333.0    NaN
1988     193.0  294.0
URS    8
USA    2
dtype: int64
